In [33]:
import pandas as pd
import numpy as np
import pandas_profiling as pf
import matplotlib.pyplot as plt
from category_encoders import OrdinalEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.neighbors import KNeighborsClassifier

In [2]:
df1 = pd.read_csv("heart_2020_cleaned.csv")
df2 = pd.read_csv("heart_disease_health_indicators_BRFSS2015.csv")

## work on the first data

In [3]:
df1.head(10)

,HeartDisease,BMI,Smoking,AlcoholDrinking,Stroke,PhysicalHealth,MentalHealth,DiffWalking,Sex,AgeCategory,Race,Diabetic,PhysicalActivity,GenHealth,SleepTime,Asthma,KidneyDisease,SkinCancer
0,No,16.60,Yes,No,No,3.0,30.0,No,Female,55-59,White,Yes,Yes,Very good,5.0,Yes,No,Yes
1,No,20.34,No,No,Yes,0.0,0.0,No,Female,80 or older,White,No,Yes,Very good,7.0,No,No,No
2,No,26.58,Yes,No,No,20.0,30.0,No,Male,65-69,White,Yes,Yes,Fair,8.0,Yes,No,No
3,No,24.21,No,No,No,0.0,0.0,No,Female,75-79,White,No,No,Good,6.0,No,No,Yes
4,No,23.71,No,No,No,28.0,0.0,Yes,Female,40-44,White,No,Yes,Very good,8.0,No,No,No
5,Yes,28.87,Yes,No,No,6.0,0.0,Yes,Female,75-79,Black,No,No,Fair,12.0,No,No,No
6,No,21.63,No,No,No,15.0,0.0,No,Female,70-74,White,No,Yes,Fair,4.0,Yes,No,Yes
7,No,31.64,Yes,No,No,5.0,0.0,Yes,Female,80 or older,White,Yes,No,Good,9.0,Yes,No,No
8,No,26.45,No,No,No,0.0,0.0,No,Female,80 or older,White,"No, borderline diabetes",No,Fair,5.0,No,Yes,No
9,No,40.69,No,No,No,0.0,0.0,Yes,Male,65-69,White,No,Yes,Good,10.0,No,No,No


In [4]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 319795 entries, 0 to 319794
Data columns (total 18 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   HeartDisease      319795 non-null  object 
 1   BMI               319795 non-null  float64
 2   Smoking           319795 non-null  object 
 3   AlcoholDrinking   319795 non-null  object 
 4   Stroke            319795 non-null  object 
 5   PhysicalHealth    319795 non-null  float64
 6   MentalHealth      319795 non-null  float64
 7   DiffWalking       319795 non-null  object 
 8   Sex               319795 non-null  object 
 9   AgeCategory       319795 non-null  object 
 10  Race              319795 non-null  object 
 11  Diabetic          319795 non-null  object 
 12  PhysicalActivity  319795 non-null  object 
 13  GenHealth         319795 non-null  object 
 14  SleepTime         319795 non-null  float64
 15  Asthma            319795 non-null  object 
 16  KidneyDisease     31

In [5]:
dropColumns = ['PhysicalHealth','MentalHealth','Race','GenHealth','SkinCancer','SleepTime','KidneyDisease']
df1.drop(columns=dropColumns,inplace=True)
df1.head(10)

,HeartDisease,BMI,Smoking,AlcoholDrinking,Stroke,DiffWalking,Sex,AgeCategory,Diabetic,PhysicalActivity,Asthma
0,No,16.60,Yes,No,No,No,Female,55-59,Yes,Yes,Yes
1,No,20.34,No,No,Yes,No,Female,80 or older,No,Yes,No
2,No,26.58,Yes,No,No,No,Male,65-69,Yes,Yes,Yes
3,No,24.21,No,No,No,No,Female,75-79,No,No,No
4,No,23.71,No,No,No,Yes,Female,40-44,No,Yes,No
5,Yes,28.87,Yes,No,No,Yes,Female,75-79,No,No,No
6,No,21.63,No,No,No,No,Female,70-74,No,Yes,Yes
7,No,31.64,Yes,No,No,Yes,Female,80 or older,Yes,No,Yes
8,No,26.45,No,No,No,No,Female,80 or older,"No, borderline diabetes",No,No
9,No,40.69,No,No,No,Yes,Male,65-69,No,Yes,No


In [6]:
df1['Diabetic'].unique()

array(['Yes', 'No', 'No, borderline diabetes', 'Yes (during pregnancy)'],
      dtype=object)

In [7]:
df1['Diabetic'].replace(['Yes (during pregnancy)','No, borderline diabetes'],['No','pre diabetes'],inplace=True)
df1['Diabetic'].unique()

array(['Yes', 'No', 'pre diabetes'], dtype=object)

## work on the second data

In [8]:
df2.head(10)

,HeartDiseaseorAttack,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,Diabetes,PhysActivity,Fruits,...,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
0,0.0,1.0,1.0,1.0,40.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,5.0,18.0,15.0,1.0,0.0,9.0,4.0,3.0
1,0.0,0.0,0.0,0.0,25.0,1.0,0.0,0.0,1.0,0.0,...,0.0,1.0,3.0,0.0,0.0,0.0,0.0,7.0,6.0,1.0
2,0.0,1.0,1.0,1.0,28.0,0.0,0.0,0.0,0.0,1.0,...,1.0,1.0,5.0,30.0,30.0,1.0,0.0,9.0,4.0,8.0
3,0.0,1.0,0.0,1.0,27.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,0.0,0.0,0.0,0.0,11.0,3.0,6.0
4,0.0,1.0,1.0,1.0,24.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,3.0,0.0,0.0,0.0,11.0,5.0,4.0
5,0.0,1.0,1.0,1.0,25.0,1.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,0.0,2.0,0.0,1.0,10.0,6.0,8.0
6,0.0,1.0,0.0,1.0,30.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,3.0,0.0,14.0,0.0,0.0,9.0,6.0,7.0
7,0.0,1.0,1.0,1.0,25.0,1.0,0.0,0.0,1.0,0.0,...,1.0,0.0,3.0,0.0,0.0,1.0,0.0,11.0,4.0,4.0
8,1.0,1.0,1.0,1.0,30.0,1.0,0.0,2.0,0.0,1.0,...,1.0,0.0,5.0,30.0,30.0,1.0,0.0,9.0,5.0,1.0
9,0.0,0.0,0.0,1.0,24.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,2.0,0.0,0.0,0.0,1.0,8.0,4.0,3.0


In [9]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 253680 entries, 0 to 253679
Data columns (total 22 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   HeartDiseaseorAttack  253680 non-null  float64
 1   HighBP                253680 non-null  float64
 2   HighChol              253680 non-null  float64
 3   CholCheck             253680 non-null  float64
 4   BMI                   253680 non-null  float64
 5   Smoker                253680 non-null  float64
 6   Stroke                253680 non-null  float64
 7   Diabetes              253680 non-null  float64
 8   PhysActivity          253680 non-null  float64
 9   Fruits                253680 non-null  float64
 10  Veggies               253680 non-null  float64
 11  HvyAlcoholConsump     253680 non-null  float64
 12  AnyHealthcare         253680 non-null  float64
 13  NoDocbcCost           253680 non-null  float64
 14  GenHlth               253680 non-null  float64
 15  

In [10]:
dropColumns = ['HighChol','Fruits', 'Veggies', 'AnyHealthcare', 'NoDocbcCost', 'GenHlth','MentHlth', 'PhysHlth','Education','Income']
df2.drop(columns=dropColumns,inplace=True)
df2.head(10)

,HeartDiseaseorAttack,HighBP,CholCheck,BMI,Smoker,Stroke,Diabetes,PhysActivity,HvyAlcoholConsump,DiffWalk,Sex,Age
0,0.0,1.0,1.0,40.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,9.0
1,0.0,0.0,0.0,25.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,7.0
2,0.0,1.0,1.0,28.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,9.0
3,0.0,1.0,1.0,27.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,11.0
4,0.0,1.0,1.0,24.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,11.0
5,0.0,1.0,1.0,25.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,10.0
6,0.0,1.0,1.0,30.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0
7,0.0,1.0,1.0,25.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,11.0
8,1.0,1.0,1.0,30.0,1.0,0.0,2.0,0.0,0.0,1.0,0.0,9.0
9,0.0,0.0,1.0,24.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,8.0


In [11]:
col = ['HeartDiseaseorAttack','Smoker','Stroke','PhysActivity','HvyAlcoholConsump','DiffWalk']
df2[col] = df2[col].replace([1,0],['Yes','No'])
df2.head(10)

,HeartDiseaseorAttack,HighBP,CholCheck,BMI,Smoker,Stroke,Diabetes,PhysActivity,HvyAlcoholConsump,DiffWalk,Sex,Age
0,No,1.0,1.0,40.0,Yes,No,0.0,No,No,Yes,0.0,9.0
1,No,0.0,0.0,25.0,Yes,No,0.0,Yes,No,No,0.0,7.0
2,No,1.0,1.0,28.0,No,No,0.0,No,No,Yes,0.0,9.0
3,No,1.0,1.0,27.0,No,No,0.0,Yes,No,No,0.0,11.0
4,No,1.0,1.0,24.0,No,No,0.0,Yes,No,No,0.0,11.0
5,No,1.0,1.0,25.0,Yes,No,0.0,Yes,No,No,1.0,10.0
6,No,1.0,1.0,30.0,Yes,No,0.0,No,No,No,0.0,9.0
7,No,1.0,1.0,25.0,Yes,No,0.0,Yes,No,Yes,0.0,11.0
8,Yes,1.0,1.0,30.0,Yes,No,2.0,No,No,Yes,0.0,9.0
9,No,0.0,1.0,24.0,No,No,0.0,No,No,No,1.0,8.0


In [12]:
df2['Sex'] = df2['Sex'].replace([1,0],['Male','Female'])
df2['Age'] = df2['Age'].replace([1,2,3,4,5,6,7,8,9,10,11,12,13]
                                ,['18-24','25-29','30-34','35-39','40-44','45-49','50-54','55-59','60-64','65-69','70-74','75-79','80 or older'])
df2.head(10)

,HeartDiseaseorAttack,HighBP,CholCheck,BMI,Smoker,Stroke,Diabetes,PhysActivity,HvyAlcoholConsump,DiffWalk,Sex,Age
0,No,1.0,1.0,40.0,Yes,No,0.0,No,No,Yes,Female,60-64
1,No,0.0,0.0,25.0,Yes,No,0.0,Yes,No,No,Female,50-54
2,No,1.0,1.0,28.0,No,No,0.0,No,No,Yes,Female,60-64
3,No,1.0,1.0,27.0,No,No,0.0,Yes,No,No,Female,70-74
4,No,1.0,1.0,24.0,No,No,0.0,Yes,No,No,Female,70-74
5,No,1.0,1.0,25.0,Yes,No,0.0,Yes,No,No,Male,65-69
6,No,1.0,1.0,30.0,Yes,No,0.0,No,No,No,Female,60-64
7,No,1.0,1.0,25.0,Yes,No,0.0,Yes,No,Yes,Female,70-74
8,Yes,1.0,1.0,30.0,Yes,No,2.0,No,No,Yes,Female,60-64
9,No,0.0,1.0,24.0,No,No,0.0,No,No,No,Male,55-59


In [13]:
df2['Diabetes'] = df2['Diabetes'].replace([2,1,0],['Yes','pre diabetes','No'])
df2.head(10)

,HeartDiseaseorAttack,HighBP,CholCheck,BMI,Smoker,Stroke,Diabetes,PhysActivity,HvyAlcoholConsump,DiffWalk,Sex,Age
0,No,1.0,1.0,40.0,Yes,No,No,No,No,Yes,Female,60-64
1,No,0.0,0.0,25.0,Yes,No,No,Yes,No,No,Female,50-54
2,No,1.0,1.0,28.0,No,No,No,No,No,Yes,Female,60-64
3,No,1.0,1.0,27.0,No,No,No,Yes,No,No,Female,70-74
4,No,1.0,1.0,24.0,No,No,No,Yes,No,No,Female,70-74
5,No,1.0,1.0,25.0,Yes,No,No,Yes,No,No,Male,65-69
6,No,1.0,1.0,30.0,Yes,No,No,No,No,No,Female,60-64
7,No,1.0,1.0,25.0,Yes,No,No,Yes,No,Yes,Female,70-74
8,Yes,1.0,1.0,30.0,Yes,No,Yes,No,No,Yes,Female,60-64
9,No,0.0,1.0,24.0,No,No,No,No,No,No,Male,55-59


In [14]:
df2.rename(columns={'HeartDiseaseorAttack': 'HeartDisease'
                    ,'Smoker': 'Smoking'
                    ,'HvyAlcoholConsump': 'AlcoholDrinking'
                    ,'DiffWalk': 'DiffWalking'
                    ,'Age': 'AgeCategory'
                    ,'Diabetes': 'Diabetic'
                    ,'PhysActivity': 'PhysicalActivity'}, inplace = True)
df2.head()

,HeartDisease,HighBP,CholCheck,BMI,Smoking,Stroke,Diabetic,PhysicalActivity,AlcoholDrinking,DiffWalking,Sex,AgeCategory
0,No,1.0,1.0,40.0,Yes,No,No,No,No,Yes,Female,60-64
1,No,0.0,0.0,25.0,Yes,No,No,Yes,No,No,Female,50-54
2,No,1.0,1.0,28.0,No,No,No,No,No,Yes,Female,60-64
3,No,1.0,1.0,27.0,No,No,No,Yes,No,No,Female,70-74
4,No,1.0,1.0,24.0,No,No,No,Yes,No,No,Female,70-74


## work on the general DATA

In [22]:
df = pd.concat([df1,df2],ignore_index=True)
df.head(10)

,HeartDisease,BMI,Smoking,AlcoholDrinking,Stroke,DiffWalking,Sex,AgeCategory,Diabetic,PhysicalActivity,Asthma,HighBP,CholCheck
0,No,16.60,Yes,No,No,No,Female,55-59,Yes,Yes,Yes,NaN,NaN
1,No,20.34,No,No,Yes,No,Female,80 or older,No,Yes,No,NaN,NaN
2,No,26.58,Yes,No,No,No,Male,65-69,Yes,Yes,Yes,NaN,NaN
3,No,24.21,No,No,No,No,Female,75-79,No,No,No,NaN,NaN
4,No,23.71,No,No,No,Yes,Female,40-44,No,Yes,No,NaN,NaN
5,Yes,28.87,Yes,No,No,Yes,Female,75-79,No,No,No,NaN,NaN
6,No,21.63,No,No,No,No,Female,70-74,No,Yes,Yes,NaN,NaN
7,No,31.64,Yes,No,No,Yes,Female,80 or older,Yes,No,Yes,NaN,NaN
8,No,26.45,No,No,No,No,Female,80 or older,pre diabetes,No,No,NaN,NaN
9,No,40.69,No,No,No,Yes,Male,65-69,No,Yes,No,NaN,NaN


In [23]:
df

,HeartDisease,BMI,Smoking,AlcoholDrinking,Stroke,DiffWalking,Sex,AgeCategory,Diabetic,PhysicalActivity,Asthma,HighBP,CholCheck
0,No,16.60,Yes,No,No,No,Female,55-59,Yes,Yes,Yes,NaN,NaN
1,No,20.34,No,No,Yes,No,Female,80 or older,No,Yes,No,NaN,NaN
2,No,26.58,Yes,No,No,No,Male,65-69,Yes,Yes,Yes,NaN,NaN
3,No,24.21,No,No,No,No,Female,75-79,No,No,No,NaN,NaN
4,No,23.71,No,No,No,Yes,Female,40-44,No,Yes,No,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
573470,No,45.00,No,No,No,No,Male,40-44,No,No,NaN,1.0,1.0
573471,No,18.00,No,No,No,Yes,Female,70-74,Yes,No,NaN,1.0,1.0
573472,No,28.00,No,No,No,No,Female,25-29,No,Yes,NaN,0.0,1.0
573473,No,23.00,No,No,No,No,Male,50-54,No,No,NaN,1.0,1.0


In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 573475 entries, 0 to 573474
Data columns (total 13 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   HeartDisease      573475 non-null  object 
 1   BMI               573475 non-null  float64
 2   Smoking           573475 non-null  object 
 3   AlcoholDrinking   573475 non-null  object 
 4   Stroke            573475 non-null  object 
 5   DiffWalking       573475 non-null  object 
 6   Sex               573475 non-null  object 
 7   AgeCategory       573475 non-null  object 
 8   Diabetic          573475 non-null  object 
 9   PhysicalActivity  573475 non-null  object 
 10  Asthma            319795 non-null  object 
 11  HighBP            253680 non-null  float64
 12  CholCheck         253680 non-null  float64
dtypes: float64(3), object(10)
memory usage: 56.9+ MB


In [25]:
def bmi2cat(i) :
    if i < 18.5 :
        return('underweight')
    elif i < 25 :
        return('healthy weight')
    elif i < 30 :
        return('overweight')
    elif i < 35 :
        return('obesity')
    elif i < 40 :
        return('over obesity')
    elif i < 50 :
        return('super obesity')
    else:
        return('super super obesity')

In [26]:
df['BMI']=df['BMI'].apply(bmi2cat)
df.head()

,HeartDisease,BMI,Smoking,AlcoholDrinking,Stroke,DiffWalking,Sex,AgeCategory,Diabetic,PhysicalActivity,Asthma,HighBP,CholCheck
0,No,underweight,Yes,No,No,No,Female,55-59,Yes,Yes,Yes,NaN,NaN
1,No,healthy weight,No,No,Yes,No,Female,80 or older,No,Yes,No,NaN,NaN
2,No,overweight,Yes,No,No,No,Male,65-69,Yes,Yes,Yes,NaN,NaN
3,No,healthy weight,No,No,No,No,Female,75-79,No,No,No,NaN,NaN
4,No,healthy weight,No,No,No,Yes,Female,40-44,No,Yes,No,NaN,NaN


In [27]:
df['Asthma'].replace(['Yes','No'],[2,1],inplace=True)
df['HighBP'].replace([1,0],[2,1],inplace=True)
df['CholCheck'].replace([1,0],[2,1],inplace=True)

In [28]:
df.head()

,HeartDisease,BMI,Smoking,AlcoholDrinking,Stroke,DiffWalking,Sex,AgeCategory,Diabetic,PhysicalActivity,Asthma,HighBP,CholCheck
0,No,underweight,Yes,No,No,No,Female,55-59,Yes,Yes,2.0,NaN,NaN
1,No,healthy weight,No,No,Yes,No,Female,80 or older,No,Yes,1.0,NaN,NaN
2,No,overweight,Yes,No,No,No,Male,65-69,Yes,Yes,2.0,NaN,NaN
3,No,healthy weight,No,No,No,No,Female,75-79,No,No,1.0,NaN,NaN
4,No,healthy weight,No,No,No,Yes,Female,40-44,No,Yes,1.0,NaN,NaN


In [29]:
oe = OrdinalEncoder()

In [30]:
df3 = df.copy(deep=1)

In [31]:
col = ['HeartDisease','BMI','Smoking','AlcoholDrinking','Stroke','DiffWalking','Sex','AgeCategory','Diabetic','PhysicalActivity']
df3[col] = oe.fit_transform(df3[col])

In [32]:
df3

,HeartDisease,BMI,Smoking,AlcoholDrinking,Stroke,DiffWalking,Sex,AgeCategory,Diabetic,PhysicalActivity,Asthma,HighBP,CholCheck
0,1,1,1,1,1,1,1,1,1,1,2.0,NaN,NaN
1,1,2,2,1,2,1,1,2,2,1,1.0,NaN,NaN
2,1,3,1,1,1,1,2,3,1,1,2.0,NaN,NaN
3,1,2,2,1,1,1,1,4,2,2,1.0,NaN,NaN
4,1,2,2,1,1,2,1,5,2,1,1.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
573470,1,5,2,1,1,1,2,5,2,2,NaN,2.0,2.0
573471,1,1,2,1,1,2,1,6,1,2,NaN,2.0,2.0
573472,1,3,2,1,1,1,1,13,2,1,NaN,1.0,2.0
573473,1,2,2,1,1,1,2,8,2,2,NaN,2.0,2.0


### impute Asthma

In [34]:
Asthma = df3.drop(columns=['HighBP','CholCheck'])
knn = KNeighborsClassifier(n_neighbors=2)
imputer = IterativeImputer(estimator=knn)
imputed1 = imputer.fit_transform(Asthma)
imp1 = pd.DataFrame(imputed1[:,-1], columns = ['Asthma']).astype(int)
imp1

,Asthma
0,2
1,1
2,2
3,1
4,1
...,...
573470,1
573471,1
573472,1
573473,1


### impute HighBP

In [35]:
HighBP = df3.drop(columns=['Asthma','CholCheck'])
knn2 = KNeighborsClassifier(n_neighbors=2)
imputer2 = IterativeImputer(estimator=knn2)
imputed2 = imputer.fit_transform(HighBP)
imp2 = pd.DataFrame(imputed2[:,-1], columns = ['HighBP']).astype(int)
imp2

,HighBP
0,1
1,2
2,2
3,1
4,1
...,...
573470,2
573471,2
573472,1
573473,2


### impute CholCheck

In [36]:
CholCheck = df3.drop(columns=['Asthma','HighBP'])
knn3 = KNeighborsClassifier(n_neighbors=2)
imputer3 = IterativeImputer(estimator=knn3)
imputed3 = imputer.fit_transform(CholCheck)
imp3 = pd.DataFrame(imputed3[:,-1], columns = ['CholCheck']).astype(int)
imp3

,CholCheck
0,2
1,2
2,2
3,2
4,1
...,...
573470,2
573471,2
573472,2
573473,2


In [37]:
df3['Asthma'] = imp1['Asthma']
df3['HighBP'] = imp2['HighBP']
df3['CholCheck'] = imp3['CholCheck']
df3

,HeartDisease,BMI,Smoking,AlcoholDrinking,Stroke,DiffWalking,Sex,AgeCategory,Diabetic,PhysicalActivity,Asthma,HighBP,CholCheck
0,1,1,1,1,1,1,1,1,1,1,2,1,2
1,1,2,2,1,2,1,1,2,2,1,1,2,2
2,1,3,1,1,1,1,2,3,1,1,2,2,2
3,1,2,2,1,1,1,1,4,2,2,1,1,2
4,1,2,2,1,1,2,1,5,2,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
573470,1,5,2,1,1,1,2,5,2,2,1,2,2
573471,1,1,2,1,1,2,1,6,1,2,1,2,2
573472,1,3,2,1,1,1,1,13,2,1,1,1,2
573473,1,2,2,1,1,1,2,8,2,2,1,2,2


In [39]:
df3[col] = oe.inverse_transform(df3[col])

In [40]:
df3

,HeartDisease,BMI,Smoking,AlcoholDrinking,Stroke,DiffWalking,Sex,AgeCategory,Diabetic,PhysicalActivity,Asthma,HighBP,CholCheck
0,No,underweight,Yes,No,No,No,Female,55-59,Yes,Yes,2,1,2
1,No,healthy weight,No,No,Yes,No,Female,80 or older,No,Yes,1,2,2
2,No,overweight,Yes,No,No,No,Male,65-69,Yes,Yes,2,2,2
3,No,healthy weight,No,No,No,No,Female,75-79,No,No,1,1,2
4,No,healthy weight,No,No,No,Yes,Female,40-44,No,Yes,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
573470,No,super obesity,No,No,No,No,Male,40-44,No,No,1,2,2
573471,No,underweight,No,No,No,Yes,Female,70-74,Yes,No,1,2,2
573472,No,overweight,No,No,No,No,Female,25-29,No,Yes,1,1,2
573473,No,healthy weight,No,No,No,No,Male,50-54,No,No,1,2,2


In [41]:
df3['Asthma'].replace([2,1],['Yes','No'],inplace=True)
df3['HighBP'].replace([2,1],['Yes','No'],inplace=True)
df3['CholCheck'].replace([2,1],['Yes','No'],inplace=True)
df3

,HeartDisease,BMI,Smoking,AlcoholDrinking,Stroke,DiffWalking,Sex,AgeCategory,Diabetic,PhysicalActivity,Asthma,HighBP,CholCheck
0,No,underweight,Yes,No,No,No,Female,55-59,Yes,Yes,Yes,No,Yes
1,No,healthy weight,No,No,Yes,No,Female,80 or older,No,Yes,No,Yes,Yes
2,No,overweight,Yes,No,No,No,Male,65-69,Yes,Yes,Yes,Yes,Yes
3,No,healthy weight,No,No,No,No,Female,75-79,No,No,No,No,Yes
4,No,healthy weight,No,No,No,Yes,Female,40-44,No,Yes,No,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...
573470,No,super obesity,No,No,No,No,Male,40-44,No,No,No,Yes,Yes
573471,No,underweight,No,No,No,Yes,Female,70-74,Yes,No,No,Yes,Yes
573472,No,overweight,No,No,No,No,Female,25-29,No,Yes,No,No,Yes
573473,No,healthy weight,No,No,No,No,Male,50-54,No,No,No,Yes,Yes


In [42]:
prof = pf.ProfileReport(df3)
prof.to_file(output_file='general.html')

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [43]:
df3.to_csv('first general DATA(KNN).csv')